## Part 1 - Synthetic Dataset Generation

In [1]:
!pip install -q transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.7 MB/s eta 0:00:00:00:0100:01


In [2]:
import time
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import pandas as pd

2025-07-24 21:14:01.828354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753391642.098353      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753391642.178463      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Path to the locally mounted model
model_path = '/kaggle/input/phi/transformers/2/1'

# Load tokenizer and model from local files
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
torch_dtype=torch.bfloat16
model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    local_files_only=True, 
    device_map="auto", 
    torch_dtype=torch.bfloat16)

generator = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [4]:
# Method to parse output of mistral, which is a giant string of json
def parse_json_from_output(text):
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        # Try to extract just the JSON block
        json_str_match = re.search(r"\[.*?\]", text, re.DOTALL)
        if json_str_match:
            try:
                return json.loads(json_str_match.group(0))
            except json.JSONDecodeError:
                return []
        else:
            return []

def parse_json_from_output(text):
    try:
        start = text.index("[")
        end = text.rindex("]") + 1
        return json.loads(text[start:end])
    except Exception as e:
        print("Failed to parse JSON:", e)
        return []

In [5]:
prompt = """
You are a helpful assistant.

Output a **valid JSON array only**. Do not include any explanation or text outside the array.

Generate 5 fictional businesses. Each item must include:
- "business_description": a one-sentence description
- "domain_name": a plausible and unique domain name

Output format:
[
  {"business_description": "...", "domain_name": "..."},
  ...
]
"""
prompt = """
You are a helpful assistant. Generate exactly 5 fictional business entries in the following JSON format:

[
  {
    "business_description": "A mobile app that helps people find local farmers markets.",
    "domain_name": "freshfinds.app"
  },
  {
    "business_description": "An online subscription service for eco-friendly cleaning supplies.",
    "domain_name": "greenbundle.com"
  },
  ...
]

Output a **valid JSON array only**. Do not include any explanation or text outside the array. Ensure all domain names are realistic and match the description.
"""

all_data = []
for i in range(1):  # or however many batches you want
    start = time.time()

    output = generator(
        prompt,
        max_new_tokens=400,
        do_sample=True,
        temperature=0.8,
        return_full_text=False
    )[0]["generated_text"]
    print(output)
    records = parse_json_from_output(output)
    all_data.extend(records)

    duration = time.time() - start
    print(f"Batch {i+1} took {duration:.2f} seconds and generated {len(records)} records.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


## INPUT

##OUTPUT
[
  {
    "business_description": "A mobile app that connects users with pet sitters in their area.",
    "domain_name": "petsitters.com"
  },
  {
    "business_description": "A subscription service for personalized meal planning based on dietary restrictions.",
    "domain_name": "mealdiet.com"
  },
  {
    "business_description": "A platform for booking yoga and meditation classes in different cities.",
    "domain_name": "mindful.com"
  },
  {
    "business_description": "An online store for second-hand clothing and accessories.",
    "domain_name": "thriftify.com"
  },
  {
    "business_description": "A mobile app that helps users track their water intake and reminds them to stay hydrated.",
    "domain_name": "waterlogged.com"
  }
]

Batch 1 took 139.62 seconds and generated 5 records.


In [6]:
df_all = pd.DataFrame(all_data).drop_duplicates(subset="domain_name")
df_all.to_csv("domain_names_with_descriptions.csv", index=False)
df_all.head()

,business_description,domain_name
0,A mobile app that connects users with pet sitt...,petsitters.com
1,A subscription service for personalized meal p...,mealdiet.com
2,A platform for booking yoga and meditation cla...,mindful.com
3,An online store for second-hand clothing and a...,thriftify.com
4,A mobile app that helps users track their wate...,waterlogged.com
